In [3]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from plaid import ApiClient, Configuration
from plaid.api.plaid_api import PlaidApi
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest

# ----- SIMPLE: hardcode .env location -----
ENV_PATH = Path("C:/Users/kosis/Downloads/Automation/spending-dashboard/scripts/.env")

if not ENV_PATH.exists():
    raise SystemExit(f".env NOT FOUND at {ENV_PATH}")

# Load the environment variables
load_dotenv(ENV_PATH)

PLAID_ENV = os.getenv("PLAID_ENV", "production").lower()
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SECRET")

if not PLAID_CLIENT_ID or not PLAID_SECRET:
    raise SystemExit("Missing PLAID_CLIENT_ID or PLAID_SECRET in .env")

# Set Plaid environment URL
base_urls = {
    "production": "https://production.plaid.com",
    "sandbox": "https://sandbox.plaid.com",
    "development": "https://development.plaid.com",
}
base_url = base_urls.get(PLAID_ENV, base_urls["production"])

config = Configuration(
    host=base_url,
    api_key={
        "clientId": PLAID_CLIENT_ID,
        "secret": PLAID_SECRET,
    }
)
client = PlaidApi(ApiClient(config))

# ----- EXCHANGE FUNCTION -----
def exchange_public_token(bank_name, public_token):
    try:
        request = ItemPublicTokenExchangeRequest(public_token=public_token)
        response = client.item_public_token_exchange(request)
        data = response.to_dict()
        access_token = data.get("access_token")
        if not access_token:
            raise RuntimeError("No access_token returned by Plaid")

        # Save or update access_tokens.json
        out_path = Path("access_tokens.json")
        tokens = json.loads(out_path.read_text()) if out_path.exists() else {}
        tokens[bank_name] = access_token
        out_path.write_text(json.dumps(tokens, indent=2), encoding="utf-8")

        print(f"✅ Saved access_token for {bank_name}: {access_token[:16]}... (keep this secret)")
    except Exception as e:
        print(f"❌ ERROR exchanging token for {bank_name}")
        print(repr(e))

# ----- EDIT THESE -----
bank_name = "Petal"
public_token = "public-production-734cb149-e8c4-41fb-9e5a-7e7d36b34dcd"

# ----- RUN IT -----
exchange_public_token(bank_name, public_token)


✅ Saved access_token for Petal: access-productio... (keep this secret)
